# stackUserViz#
*Now with more meta!*

A Jupyter notebook for collecting, processing and visualizing user activity trajectories on Stack Exchange sites and their accompanying meta sites.

- The first cell has settings and should be run before running any of the following cells
- The second cell collects questions, answers, and comments from Stack Exchange main sites for a particular user. It produces one CSV file with all questions, answer and comments
- The third cell collects questions, answers, and comments from Stack Exchange meta sites for a particular user. It produces one CSV file with all questions, answer and comments 
- The fourth cell collects reputation change events, orders the data, computes a running reputation score, and writes them to a CSV file
- The fifth cell loads and processes the posting activity and reputation event files before summarizing and plotting the data. A short descriptive statistical summary is produced and saved as a text file and then a plot is generated showing posting activity as a stacked bar chart with questions, answers and comments, and reputation score as a line graph. Both datasets are aggregated and plotted monthly to show a continuous timeline that visualizes a user's activity on the platform as a trajectory

To use these scripts, you will need a Stack Exchange API key that you put in a plain text file named stackApiKey.txt and put in the same directory as this notebook.

In [ ]:
# Setup: Always run this first!

from stackapi import StackAPI
from datetime import datetime, timedelta
import csv
import pandas as pd
import sys
import os
import glob
import d6tstack.combine_csv
import time
import os
import numpy as np
from mpl_axes_aligner import align
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

# Choose community
community = input("Which StackExchange community would you like to work with? ")

# Choose a user
userId = input("Which "+community+" user ID would you like to work with? ")

keyfile = open('stackApiKey.txt', 'r') 
apiKey = keyfile.read() 

SITE = StackAPI(community, key=apiKey)
SITE.page_size = 100
SITE.max_pages = 10000

stackLaunch = datetime(2008, 9, 22)

################################
# Edit collection settings here:
# Set the period for collection
startDate = stackLaunch
#startDate = datetime(2008, 9, 22)

endDate = datetime.today()
#endDate = datetime(2020, 9, 22)

# Set the length of each api call in days
step = 365
# Set a pause length in seconds to not violate the data cap on the API
pause = 2
################################

In [ ]:
# Collect main site questions, answers and comments

# Items to be dropped from datasets
question_drop = ['owner_profile_image',
                 'owner_link',
                 'migrated_from_other_site_styling_tag_background_color',
                 'migrated_from_other_site_styling_tag_foreground_color',
                 'migrated_from_other_site_styling_link_color',
                 'migrated_from_other_site_related_sites',
                 'migrated_from_other_site_markdown_extensions',
                 'migrated_from_other_site_launch_date',
                 'migrated_from_other_site_open_beta_date',
                 'migrated_from_other_site_site_state',
                 'migrated_from_other_site_high_resolution_icon_url',
                 'migrated_from_other_site_twitter_account',
                 'migrated_from_other_site_favicon_url',
                 'migrated_from_other_site_icon_url',
                 'migrated_from_other_site_audience',
                 'migrated_from_other_site_site_url',
                 'migrated_from_other_site_api_site_parameter',
                 'migrated_from_other_site_logo_url',
                 'migrated_from_other_site_name',
                 'migrated_from_other_site_site_type',
                 'migrated_from_other_site_closed_beta_date',
                 'migrated_from_other_site_aliases']
comment_drop = ['owner_profile_image',
                'owner_link',
                'reply_to_user_profile_image',
                'reply_to_user_link']
answer_drop = ['owner_profile_image',
               'owner_link']

# Get questions
questions = SITE.fetch('users/'+userId+'/questions', filter='withbody')
df = pd.json_normalize(questions['items'],sep='_')

# Stop if no data
if len(df) < 2:
    sys.exit('No activty')

# Clean up
df.drop(question_drop, inplace=True, axis=1, errors='ignore')
if 'last_activity_date' in df.columns:
    df['last_activity_date'] = pd.to_datetime(df['last_activity_date'], unit='s')
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
if 'last_edit_date' in df.columns:
    df['last_edit_date'] = pd.to_datetime(df['last_edit_date'], unit='s')
if 'closed_date' in df.columns:
    df['closed_date'] = pd.to_datetime(df['closed_date'], unit='s')
if 'migrated_from_on_date' in df.columns:
    df['migrated_from_on_date'] = pd.to_datetime(df['migrated_from_on_date'], unit='s')
# Write processed trajectories to csv
df['post_type']='question'
df.to_csv(community+'_'+userId+'_questions.csv')
print('Questions collected')

# Get comments
comments = SITE.fetch('users/'+userId+'/comments', filter='withbody')
df = pd.json_normalize(comments['items'],sep='_')
# Clean up
df.drop(comment_drop, inplace=True, axis=1, errors='ignore')
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
# Write processed trajectories to csv
df['post_type']='comment'
df.to_csv(community+'_'+userId+'_comments.csv')
print('Comments collected')
        
# Get answers
answers = SITE.fetch('users/'+userId+'/answers', filter='withbody')
df = pd.json_normalize(answers['items'],sep='_')
# Clean up
df.drop(answer_drop, inplace=True, axis=1, errors='ignore')
if 'last_activity_date' in df.columns:
    df['last_activity_date'] = pd.to_datetime(df['last_activity_date'], unit='s')
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
if 'last_edit_date' in df.columns:
    df['last_edit_date'] = pd.to_datetime(df['last_edit_date'], unit='s')
if 'community_owned_date' in df.columns:
    df['community_owned_date'] = pd.to_datetime(df['community_owned_date'], unit='s')
# Write processed trajectories to csv
df['post_type']='answer'
df.to_csv(community+'_'+userId+'_answers.csv')
print('Answers collected')

#Create a combined file
files = [community+'_'+userId+'_questions.csv', 
         community+'_'+userId+'_comments.csv',
         community+'_'+userId+'_answers.csv']
combined = d6tstack.combine_csv.CombinerCSV(files).to_csv_combine(community+'_'+userId+'.csv')
df = pd.read_csv(community+'_'+userId+'.csv', index_col=[0], dtype=object)
df.drop({'filepath', 'filename'}, inplace=True, axis=1, errors='ignore')
df.to_csv(community+'_'+userId+'.csv', index=False)
for file in files:
    os.remove(file)

print('Main site posts collected')

In [ ]:
# Collect meta site questions, answers and comments

SITE = StackAPI('meta.'+community, key=apiKey)

# Items to be dropped from datasets
question_drop = ['owner_profile_image',
                 'owner_link',
                 'migrated_from_other_site_styling_tag_background_color',
                 'migrated_from_other_site_styling_tag_foreground_color',
                 'migrated_from_other_site_styling_link_color',
                 'migrated_from_other_site_related_sites',
                 'migrated_from_other_site_markdown_extensions',
                 'migrated_from_other_site_launch_date',
                 'migrated_from_other_site_open_beta_date',
                 'migrated_from_other_site_site_state',
                 'migrated_from_other_site_high_resolution_icon_url',
                 'migrated_from_other_site_twitter_account',
                 'migrated_from_other_site_favicon_url',
                 'migrated_from_other_site_icon_url',
                 'migrated_from_other_site_audience',
                 'migrated_from_other_site_site_url',
                 'migrated_from_other_site_api_site_parameter',
                 'migrated_from_other_site_logo_url',
                 'migrated_from_other_site_name',
                 'migrated_from_other_site_site_type',
                 'migrated_from_other_site_closed_beta_date',
                 'migrated_from_other_site_aliases']
comment_drop = ['owner_profile_image',
                'owner_link',
                'reply_to_user_profile_image',
                'reply_to_user_link']
answer_drop = ['owner_profile_image',
               'owner_link']

# Get questions
questions = SITE.fetch('users/'+userId+'/questions', filter='withbody')
df = pd.json_normalize(questions['items'],sep='_')
# Clean up
df.drop(question_drop, inplace=True, axis=1, errors='ignore')
if 'last_activity_date' in df.columns:
    df['last_activity_date'] = pd.to_datetime(df['last_activity_date'], unit='s')
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
if 'last_edit_date' in df.columns:
    df['last_edit_date'] = pd.to_datetime(df['last_edit_date'], unit='s')
if 'closed_date' in df.columns:
    df['closed_date'] = pd.to_datetime(df['closed_date'], unit='s')
if 'migrated_from_on_date' in df.columns:
    df['migrated_from_on_date'] = pd.to_datetime(df['migrated_from_on_date'], unit='s')
# Write processed trajectories to csv
df['post_type']='question'
df.to_csv(community+'_'+userId+'_questions.csv')
print('Questions collected')

# Get comments
comments = SITE.fetch('users/'+userId+'/comments', filter='withbody')
df = pd.json_normalize(comments['items'],sep='_')
# Clean up
df.drop(comment_drop, inplace=True, axis=1, errors='ignore')
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
# Write processed trajectories to csv
df['post_type']='comment'
df.to_csv(community+'_'+userId+'_comments.csv')
print('Comments collected')
        
# Get answers
answers = SITE.fetch('users/'+userId+'/answers', filter='withbody')
df = pd.json_normalize(answers['items'],sep='_')
# Clean up
df.drop(answer_drop, inplace=True, axis=1, errors='ignore')
if 'last_activity_date' in df.columns:
    df['last_activity_date'] = pd.to_datetime(df['last_activity_date'], unit='s')
if 'creation_date' in df.columns:
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
if 'last_edit_date' in df.columns:
    df['last_edit_date'] = pd.to_datetime(df['last_edit_date'], unit='s')
if 'community_owned_date' in df.columns:
    df['community_owned_date'] = pd.to_datetime(df['community_owned_date'], unit='s')
# Write processed trajectories to csv
df['post_type']='answer'
df.to_csv(community+'_'+userId+'_answers.csv')
print('Answers collected')

#Create a combined file
files = [community+'_'+userId+'_questions.csv', 
         community+'_'+userId+'_comments.csv',
         community+'_'+userId+'_answers.csv']

combined = d6tstack.combine_csv.CombinerCSV(files).to_csv_combine('meta_'+community+'_'+userId+'.csv')
df = pd.read_csv('meta_'+community+'_'+userId+'.csv', index_col=[0], dtype=object)
df.drop({'filepath', 'filename'}, inplace=True, axis=1, errors='ignore')
df.to_csv('meta_'+community+'_'+userId+'.csv', index=False)

#Remove temporary files
for file in files:
    os.remove(file)
    
#Remove combined file if number of rows is below threshold    
if len(df) < 10:
    os.remove('meta_'+community+'_'+userId+'.csv')
    print('Too little meta activity')
else:
    print('meta site posts collected')

In [ ]:
# Collect reputation events

stepDate = startDate + timedelta(days=step)

# Collect reputation events
while stepDate < endDate:
    events = SITE.fetch('users/'+userId+'/reputation-history', fromdate=startDate, todate=stepDate)
    df = pd.json_normalize(events['items'],sep='_')
    df.to_csv('temp_'+startDate.strftime('%Y-%m-%d')+'_'+stepDate.strftime('%Y-%m-%d')+'.csv')
    time.sleep(pause)
    print(str(startDate)+' to '+str(stepDate)+' collected')
    startDate = stepDate + timedelta(days=1)
    stepDate = startDate + timedelta(days=step)

else:
    events = SITE.fetch('users/'+userId+'/reputation-history', fromdate=startDate, todate=endDate)
    df = pd.json_normalize(events['items'],sep='_')
    df.to_csv('temp_'+startDate.strftime('%Y-%m-%d')+'_'+stepDate.strftime('%Y-%m-%d')+'.csv')
    print(str(startDate)+' to '+str(endDate)+' collected')

# Stitch the yearly files together
files = list(glob.glob('temp_*.csv'))
li = []
for filename in files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    li.append(df1)
df = pd.concat(li, axis=0, ignore_index=True)

# Clean up temporary files
files = list(glob.glob('temp_*.csv'))
for file in files:
    os.remove(file)

# Sort by creation_date or remove files if empty
if df.empty:
    print('No reputation events')

else:
    df = df.sort_values(by = 'creation_date') 
    # Create nomalized timestamp timeline
    df['time_delta'] = df['creation_date'].diff()
    # Calculate cumulative sum of reputation
    df['days'] = df['time_delta'].cumsum() / 86400
    df['reputation'] = df['reputation_change'].cumsum()
    # Convert timestamps to datetime
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
    # Write processed reputation events to csv
    df.to_csv(community+'_'+userId+'_reputation.csv')
    print('Reputation events collected')

In [ ]:
# Import user data and visualize it

if os.path.isfile(community+'_'+userId+'.csv'):
    df = pd.read_csv(community+'_'+userId+'.csv', parse_dates=['creation_date'])
else:
    sys.exit('No activty')

if os.path.isfile('meta_'+community+'_'+userId+'.csv'):
    mdf = pd.read_csv('meta_'+community+'_'+userId+'.csv')
    mdf = pd.read_csv('meta_'+community+'_'+userId+'.csv', parse_dates=['creation_date'])

if os.path.isfile(community+'_'+userId+'_reputation.csv'):
    repdf = pd.read_csv(community+'_'+userId+'_reputation.csv', parse_dates=['creation_date'])

# Descriptive stats
date = df['creation_date']
since = date.min()
print('Active since: '+str(since))

if os.path.isfile(community+'_'+userId+'_reputation.csv'):
    rep = repdf['reputation']
    max_rep = rep.max()
    print('Reputation: '+str(max_rep))

posts = df.shape[0]
print('Posts on '+community+': '+str(posts))

if os.path.isfile('meta_'+community+'_'+userId+'.csv'):
    mposts = mdf.shape[0]
    print('Posts on meta.'+community+': '+str(mposts))

dfv = df['post_type'].value_counts()
print('Post types on '+community+':')
print(dfv)

if os.path.isfile('meta_'+community+'_'+userId+'.csv'):
    mdfv = mdf['post_type'].value_counts()
    print('Post types on meta.'+community+':')
    print(mdfv)

# Plot activity and reputation on a single figure

# Process main posting data
df['month_year'] = pd.to_datetime(df['creation_date']).map(lambda dt: dt.replace(day=1))
df['month_year'] = pd.to_datetime(df['month_year']).dt.date
df1 = df.groupby(['month_year', 'post_type'])['creation_date'].count().reset_index(name='count')
df1 = df1.pivot(index='month_year', columns='post_type', values='count')
df1.index = pd.to_datetime(df1.index)
df1 = df1.resample('MS').asfreq().fillna(0)
df1 = df1.reset_index()
df1.month_year = df1.month_year.dt.date
df1.month_year = df1.month_year.astype('object')
df1[['answer', 'comment', 'question']] = df1[['answer', 'comment', 'question']].astype('int64')
df1.rename(columns={'answer': 'Answers', 'comment': 'Comments', 'question': 'Questions'}, inplace=True)

# Process meta posting data
if os.path.isfile('meta_'+community+'_'+userId+'.csv'):
    mdf['month_year'] = pd.to_datetime(mdf['creation_date']).map(lambda dt: dt.replace(day=1))
    mdf['month_year'] = pd.to_datetime(mdf['month_year']).dt.date
    mdf1 = mdf.groupby(['month_year', 'post_type'])['creation_date'].count().reset_index(name='count')
    mdf1 = mdf1.pivot(index='month_year', columns='post_type', values='count')
    mdf1.index = pd.to_datetime(mdf1.index)
    mdf1 = mdf1.resample('MS').asfreq().fillna(0)
    mdf1 = mdf1.reset_index()
    mdf1.month_year = mdf1.month_year.dt.date
    mdf1.month_year = mdf1.month_year.astype('object')
    if 'answer' not in mdf1:
        mdf1['answer'] = 0
    if 'comment' not in mdf1:
        mdf1['comment'] = 0
    if 'question' not in mdf1:
        mdf1['question'] = 0
    mdf1[['answer', 'comment', 'question']] = mdf1[['answer', 'comment', 'question']].astype('int64')
    mdf1.rename(columns={'answer': 'meta.Answers', 'comment': 'meta.Comments', 'question': 'meta.Questions'}, inplace=True)
    mdf1['Meta Posts'] = mdf1['meta.Answers'] + mdf1['meta.Comments'] + mdf1['meta.Questions']

# Process reputation data
if os.path.isfile(community+'_'+userId+'_reputation.csv'):
    repdf1 = repdf[['creation_date', 'reputation']].copy()
    repdf1['month_year'] = pd.to_datetime(repdf1['creation_date']).map(lambda dt: dt.replace(day=1))
    repdf1['month_year'] = pd.to_datetime(repdf1['month_year']).dt.date
    repdf1 = repdf1.groupby(['month_year'], sort=False)['reputation'].max()
    repdf1.index = pd.to_datetime(repdf1.index)
    repdf1 = repdf1.resample('MS').asfreq().fillna(method='ffill')
    repdf1 = repdf1.reset_index()
    repdf1.month_year = repdf1.month_year.dt.date
    repdf1.month_year = repdf1.month_year.astype('object')
    repdf1['reputation'] = repdf1['reputation'].astype('int64')
    repdf1.rename(columns={'reputation': 'Reputation'}, inplace=True)

# Create a merged dataframe, but ignore meta if empty
if not os.path.isfile('meta_'+community+'_'+userId+'.csv'):
    merged = df1.copy()
else:
    merged = pd.merge(df1, mdf1, on=['month_year'], how='outer')
    merged = pd.merge(merged, repdf1, on=['month_year'], how='outer')
    
merged.month_year = pd.to_datetime(merged.month_year)
merged = merged.sort_values('month_year').reset_index(drop=True)

# Plot the figures
fig, ax = plt.subplots(figsize = (15,4))
ax.axhline(0, color='#EEEEEE')

if not os.path.isfile('meta_'+community+'_'+userId+'.csv'):
    ax1 = merged[['Answers', 'Comments', 'Questions']].plot(kind='bar', legend=False, stacked=True, color=['#284E60', '#F99B45', '#63AAC0'], width=0.8, ax=ax)
    ax2 = merged['Reputation'].plot(legend=False, secondary_y=True, color='#D95980', linewidth=2, ax=ax)
    for ax in (ax1, ax2):
        ax.spines["top"].set_visible(False)    
        ax.spines["bottom"].set_visible(False)    
        ax.spines["right"].set_visible(False)    
        ax.spines["left"].set_visible(False)
        ax.tick_params(bottom=False, left=False, right=False)
        ax.set_axisbelow(True)
        ax.xaxis.grid(False)
        ax.ticklabel_format(useOffset=False, style='plain', axis='y')
    # Axis
    align.yaxes(ax1, 0.0, ax2, 0.0, 0.1)
    max_value = merged.index.max()
    min_value = merged.index.min()
    number_of_steps = 5
    l = np.arange(min_value, max_value+1, number_of_steps)
    ax.set(xticks=l, xticklabels=l)
    ax1.tick_params(axis='x', pad=-20)
    ax1.yaxis.grid(True, color='#EEEEEE')
    # Labels
    ax.set_xlabel('Month of Participation', labelpad=10, color='#333333')
    ax1.set_ylabel('Posts', labelpad=15, color='#333333')
    ax2.set_ylabel('Reputation Score', labelpad=15, color='#333333') 
    plt.title('Participation trajectory for '+community+' user '+userId, fontsize=15, color='#333333', loc='left', pad=5)
    # Legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    leg = ax.legend(lines1 + lines2, labels1 + labels2, frameon=False, loc='upper right', ncol=5, bbox_to_anchor=(1.1, 1.1))
    for text in leg.get_texts():
        plt.setp(text, color = '#333333')
        
else:
    ax1 = merged[['Answers', 'Comments', 'Questions']].plot(kind='bar', legend=False, stacked=True, color=['#284E60', '#F99B45', '#63AAC0'], width=0.8, ax=ax)
    ax2 = merged['Meta Posts'].plot(legend=False, color='#2a9d8f', linewidth=2, ax=ax)
    ax3 = merged['Reputation'].plot(legend=False, secondary_y=True, color='#D95980', linewidth=2, ax=ax)
    for ax in (ax1, ax2, ax3):
        ax.spines["top"].set_visible(False)    
        ax.spines["bottom"].set_visible(False)    
        ax.spines["right"].set_visible(False)    
        ax.spines["left"].set_visible(False)
        ax.tick_params(bottom=False, left=False, right=False)
        ax.set_axisbelow(True)
        ax.xaxis.grid(False)
        ax.ticklabel_format(useOffset=False, style='plain', axis='y')
    
    # Axis
    align.yaxes(ax1, 0.0, ax3, 0.0, 0.1)
    max_value = merged.index.max()
    min_value = merged.index.min()
    number_of_steps = 5
    l = np.arange(min_value, max_value+1, number_of_steps)
    ax.set(xticks=l, xticklabels=l)
    ax1.tick_params(axis='x', pad=-20)
    ax1.yaxis.grid(True, color='#EEEEEE')
    # Labels
    ax.set_xlabel('Month of Participation', labelpad=10, color='#333333')
    ax1.set_ylabel('Posts', labelpad=15, color='#333333')
    ax3.set_ylabel('Reputation Score', labelpad=15, color='#333333') 
    plt.title('Participation trajectory for '+community+' user '+userId, fontsize=15, color='#333333', loc='left', pad=5)
    # Legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines3, labels3 = ax3.get_legend_handles_labels()
    leg = ax.legend(lines1 + lines3, labels1 + labels3, frameon=False, loc='upper right', ncol=5, bbox_to_anchor=(1.1, 1.1))
    for text in leg.get_texts():
        plt.setp(text, color = '#333333')

# Save and show figure
save = plt.gcf()
plt.show()
save.savefig(community+'_'+userId+'.png')